# 0. Imports

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import re
import unicodedata
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

import evaluate

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EsdrasDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\EsdrasDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\EsdrasDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\EsdrasDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## 0.1 - Funções

In [93]:
def normalizar_texto(texto: str, formatacao: str = 'NFD', remover_especiais=True):
    """
    Função para normalizar texto, removendo acentos e caracteres especiais.
    Parâmetros:
    texto: str - texto a ser normalizado
    formatacao: str - formatação a ser utilizada para normalização. Padrão: NFD
    Retorna:
    str - texto normalizado
    Exemplo:
    string_text = 'áéíóúçãõ ção -- __ ¬ ²³£¢¬9)( " ///°°?<>) jisadsa !// 1234 /*-+,&%$#@!'
    for form in ['NFC', 'NFKC', 'NFD', 'NFKD']:
        print(normalizar_texto(string_text, form))
    """

    # check valid formatacao
    if formatacao not in ['NFC', 'NFKC', 'NFD', 'NFKD']:
        raise ValueError('formatacao must be one of: NFC, NFKC, NFD, NFKD')

    if texto:
        # convert texto to str if not
        if type(texto) is not str:
            texto = str(texto)
        nfkd = unicodedata.normalize(formatacao, texto)  # NFKD
        palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
        if remover_especiais is True:
            palavra_sem_acento = re.sub('[^a-zA-Z0-9 \\\]', '', palavra_sem_acento)

        return palavra_sem_acento.replace('  ', ' ')

    return texto

def preprocess_text_safe(text, with_space=True):
    """
    Realiza o pré-processamento de um texto, ignorando erros.

    Parâmetros:
        - text (str): Texto judicial a ser processado.

    Retorna:
        - Texto limpo e lematizado (str), ou uma string vazia se ocorrer um erro.
    """

    erros = []

    try:

        strange_words = [
            'assim',
            'parte',
            'rozana',
            'ato',
            'grande',
            'juiz djanirito souza moura',
            'cidade',
            'estado rio grande norte',
            'rio norte',
            'assinado',
            '36738671',
            'normal',
            'cod',
            'juiz',
            'ptbr',
            'lei',
            'praca sete setembro',
            'auto',
            'forma',
            'secretaria',
            'cidade alta',
            'data',
            'tributaria natal',
            'nao',
            '5902530',
            '59025275',
            'alta',
            'email',
            'rn',
            'publico',
            'valor',
            'intimese',
            'setembro',
            'norte',
            'whatsapp',
            'digitalmente',
            '2024',
            'natalrn',
            'judiciario',
            'secuniefttj',
            '59025300',
            'comarca',
            'obrig',
            'termos',
            'apos',
            '1141906',
            'estado',
            'processo',
            'rel',
            'forum fazendario',
            'forum',
            'fazendario',
            'norte',
            'telefone',
            'vara',
            'documento',
            'n1141906',
            'caso',
            'publica',
            'poder judiciario',
            'poder',
            'silva',
            'xnone',
            'prazo',
            'justica',
            'juiz djanirito souza mouro',
            'igo',
            'ate',
            'data registrada sistema',
            'sobre',
            'false',
            'sistema',
            'codigo',
            'dia',
            'direito',
            'cpc',
            'veft',
            'jusbr',
            'desde',
            'civil',
            'vara execucao fiscal',
            'juiza',
            'valores',
            'art',
            'cep',
            'conforme',
            'natal',
            'natalpraca',
            'forma lei',
            'tributaria',
            'rio',
            'praca alto',
            'acao',
            'municipio',
            'sete',
            'voltem',
            'conclusos',
            'publiquese',
            'cumprase',
            'artigo',
            'bem',
            'presente',
            'devera',
            'sendo',
            'ano',
            'inciso',
            'maria',
            'disposto',
            'ainda',
            'federal',
            'turma',
            'feito',
            'meio',
            'intimemse',
            'partir',
            'sob',
            'dje',
            'junho',
            'intimo',
            'julho',
            'manifestarse',
            'chefe',
            '59025300contato',
            '203'
        ]

        # Inicializar lematizador e stopwords
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('portuguese'))

        text = normalizar_texto(text, 'NFD', remover_especiais=True)

        # Tokenizar texto
        tokens = word_tokenize(text)
        # Remover stopwords e lematizar tokens
        tokens = [
            lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2
        ]

        # Reconstruir o texto processado
        processed_text = " ".join(tokens)

        # Remover strange_words com Regex
        pattern_re = "|".join(strange_words)

        processed_text = re.sub(
           f"\\b({pattern_re})\\b", "", processed_text, flags=re.IGNORECASE
        )

        # remover espacos duplos
        processed_text = " ".join(re.split(r"\s+", processed_text))
        if with_space is False:
            processed_text = processed_text.replace(" ", "")

        return processed_text

    except Exception as e:
        erro = str(e)
        if erro not in erros:
            erros.append(erro)
            print(f"Erro ao processar texto: {erro}")
            return ""  # Retorna texto vazio em caso de erro
        
import pandas as pd
from sklearn.model_selection import train_test_split

def separar_e_dividir_dados(df: pd.DataFrame, coluna_classe: str, n_samples_per_class: int = 500, train_frac:float = 0.7, test_frac: float = 0.15, val_frac: float = 0.15):
    """
    Separa os dados de um DataFrame com classes desbalanceadas e divide em treinamento, validação e teste.
    
    Parâmetros:
        df (pd.DataFrame): O dataframe contendo os dados.
        coluna_classe (str): O nome da coluna que contém as classes.
        min_ocorrencias (int): Número mínimo de ocorrências para uma classe ser considerada balanceada.
        
    Retorna:
        tuple: DataFrames de treinamento, validação e teste.
    """
    soma = sum([train_frac, test_frac, val_frac])
    if abs(soma - 1) > 1e-6:
        raise ValueError(f"A soma das porcentagem de split dos dados é {soma}, que não é aproximadamente igual a 1. Tolerância = 1e-6")

    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    val_df = pd.DataFrame()
    
    # Itera sobre cada classe única
    for classe, grupo in df.groupby(coluna_classe):
        if len(grupo) <= n_samples_per_class:
            # Utiliza todos os dados
            grupo = grupo.sample(frac=1, random_state=42).reset_index(drop=True)
            
            '''train_size = int(train_frac * len(grupo))
            test_size = int(test_frac * len(grupo))
            
            train_df = pd.concat(train_df, grupo[:train_size])
            test_df = pd.concat(test_df, grupo[train_size:train_size+test_size])
            val_df = pd.concat(val_df, grupo[train_size+test_size:])'''
            
        else:
            # Limita ao número de amostras por classe
            grupo = grupo.sample(n=n_samples_per_class, random_state=42)
            
            '''train_size = int(train_frac * len(grupo))
            test_size = int(test_frac * len(grupo))
            
            train_df = pd.concat(train_df, grupo[:train_size])
            test_df = pd.concat(test_df, grupo[train_size:train_size+test_size])
            val_df = pd.concat(val_df, grupo[train_size+test_size:])'''
            
        train_size = int(train_frac * len(grupo))
        test_size = int(test_frac * len(grupo))
        
        train_df = pd.concat([train_df, grupo[:train_size]])
        test_df = pd.concat([test_df, grupo[train_size:train_size+test_size]])
        val_df = pd.concat([val_df, grupo[train_size+test_size:]])

    # Faz o shuffle nos DataFrames
    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
    val_df = val_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Converte os DataFrames para Hugging Face Datasets
    train_ds = Dataset.from_pandas(train_df)
    test_ds = Dataset.from_pandas(test_df)
    val_ds = Dataset.from_pandas(val_df)
    
    # Combina num DatasetDict
    dataset_dict = DatasetDict({
        'train': train_ds,
        'validation': val_ds,
        'test': test_ds
    })

    return dataset_dict


# 1. Preparação dos dados

## 1.1 - Loading e Pré-processamento

In [94]:
df = pd.read_parquet('./data/pgm-dataset-new.parquet')
df.rename(columns={'general_classes': 'labels'}, inplace=True)

# Criando uma lista com os nomes das classes
classes = np.unique(df['labels'])

# Alterando as labels no DataFrame
id2label = {i: classe for i, classe in enumerate(classes)}
label2id = {classe: i for i, classe in enumerate(classes)}

df['labels'] = df['labels'].map(label2id)

print(f'Shape: {df.shape}')
df.head(3)

Shape: (15347, 10)


,intimacao_conteudo,processo_conteudo,rstREGEX,teorIntimacao,intimacaoPJE,processoPJE,idavisopje,setordestino,Classificacao,labels
0,,,"[{""diligencia"": ""intime-se o Município de Nata...",PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,"{""id"": ""18612632"", ""tipoComunicacao"": ""INT"", ""...","{""Numero"": ""08387198120248205001"", ""Competenci...",18612632,Procuradoria da Saude,,7
1,,,"[{""diligencia"": ""intimado acerca da obrigação ...",PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,"{""id"": ""18629258"", ""tipoComunicacao"": ""INT"", ""...","{""Numero"": ""08464942120228205001"", ""Competenci...",18629258,Procuradoria Administrativa,,0
3,,,"[{""diligencia"": ""intime-se a parte exequente p...",PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,"{""id"": ""18629665"", ""tipoComunicacao"": ""INT"", ""...","{""Numero"": ""01263147320118200001"", ""Competenci...",18629665,APOIO FISCAL,,2


In [95]:
tqdm.pandas()

df['teorIntimacao_clean'] = df['teorIntimacao'].progress_apply(preprocess_text_safe)

100%|██████████| 15347/15347 [01:14<00:00, 205.49it/s]


In [96]:
df['labels'].value_counts()

labels
2    6686
0    5401
1    1338
4    1137
7     443
5     155
6     121
3      66
Name: count, dtype: int64

## 1.2 - Split de dados

In [97]:
dataset_dict = separar_e_dividir_dados(df, 'labels', n_samples_per_class=400, train_frac=0.7, test_frac=0.15, val_frac=0.15)

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean'],
        num_rows: 1638
    })
    validation: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean'],
        num_rows: 354
    })
    test: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean'],
        num_rows: 350
    })
})

# 2. BERT Fine-Tuning

## 2.1 - Carregando o modelo pré-treinado

In [98]:
model_path = 'google-bert/bert-base-multilingual-uncased'

# Carrega o tokenizador do modelo
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Carrega o modelo para classificação
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=8,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 2.2 - Congelando Parâmetros

Congelando os parâmetros do modelo base para que todos os 110M de parâmetros não sejam modificados

In [99]:
for name, param in model.base_model.named_parameters():
    if not 'pooler' in name:
        param.requires_grad = False

## 2.3 - Pré-processamento

In [100]:
def preprocess_function(texts):
    # Retorna o texto tokenizado e truncado
    return tokenizer(texts['teorIntimacao_clean'], truncation=True)

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map: 100%|██████████| 350/350 [00:00<00:00, 1715.30 examples/s]


In [101]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1638
    })
    validation: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 354
    })
    test: Dataset({
        features: ['intimacao_conteudo', 'processo_conteudo', 'rstREGEX', 'teorIntimacao', 'intimacaoPJE', 'processoPJE', 'idavisopje', 'setordestino', 'Classificacao', 'labels', 'teorIntimacao_clean', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 350
    })
})

In [102]:
# Criando Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2.4 - Definindo métricas de avaliação

In [103]:
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')
accuracy_metric = evaluate.load('accuracy')

#clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    # Get predictions
    predictions, labels = eval_pred
    
    # Aplicando Softmax para recuperar as probabilidades
    predictions = predictions - np.max(predictions)
    probabilidades = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                   keepdims=True)
    # Predizendo a classe mais provável
    predicted_classes = np.argmax(probabilidades, axis=1)
    
    # Calcular métricas
    precision = precision_metric.compute(predictions=predicted_classes, references=labels, average='weighted')["precision"]
    recall = recall_metric.compute(predictions=predicted_classes, references=labels, average='weighted')["recall"]
    f1 = f1_metric.compute(predictions=predicted_classes, references=labels, average='weighted')["f1"]
    accuracy = accuracy_metric.compute(predictions=predicted_classes, references=labels)["accuracy"]
    
    return {'precision':precision, 'recall':recall, 'f1-score':f1, 'accuracy':accuracy}
    

## 2.5 - Treinando

In [106]:
lr = 2e-4
batch_size = 8
num_epochs = 20

training_args = TrainingArguments(
    output_dir='./models/bert-pgm-classification_teacher',
    #label_names=classes,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\EsdrasDaniel\AppData\Local\Temp\ipykernel_9412\2266952429.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,1.002900,1.145136,0.645777,0.588571,0.546659,0.588571
2,0.979400,1.046957,0.630503,0.611429,0.604815,0.611429
3,0.971100,1.053582,0.662567,0.648571,0.609570,0.648571
4,0.977900,1.048332,0.639359,0.637143,0.607691,0.637143
5,0.958800,1.085661,0.651851,0.637143,0.628577,0.637143
6,0.958300,0.998393,0.635247,0.634286,0.618856,0.634286
7,0.963500,1.055722,0.646686,0.605714,0.587864,0.605714
8,1.060600,1.046773,0.690899,0.637143,0.599284,0.637143
9,1.072900,1.025593,0.670688,0.651429,0.612639,0.651429
10,1.056900,1.024858,0.678450,0.660000,0.630796,0.660000


TrainOutput(global_step=4100, training_loss=1.0199584514338795, metrics={'train_runtime': 32576.6863, 'train_samples_per_second': 1.006, 'train_steps_per_second': 0.126, 'total_flos': 8430628971653952.0, 'train_loss': 1.0199584514338795, 'epoch': 20.0})